In [65]:
# Importing necessary libraries

# data manipulation
import numpy as np
import pandas as pd

# plots
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.graphics.gofplots import qqplot

# date library
from datetime import datetime

# time series librarires
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

# statistical test - parametric and non-parametric
from scipy.stats import pearsonr
from scipy.stats import norm
from scipy.stats import shapiro
from scipy.stats import normaltest
from scipy.stats import mannwhitneyu
from scipy.stats import kruskal
from statsmodels.tsa.stattools import adfuller

# feature selection libraries

# Preparing data for modeling

In [66]:
data = pd.read_csv('../Rossmann_Store_Sales/Data/df.csv')

In [67]:
data['Date'] = pd.to_datetime(data['Date'])

In [68]:
data.shape

(844338, 22)

In [69]:
data_columns = data.columns

In [70]:
# columns with missing values
# StoreType
# log_competition_distance
nan_values = data.isna()
nan_columns = nan_values.any()

columns_with_nan = data.columns[nan_columns].tolist()
print(columns_with_nan)

['StoreType', 'log_competition_distance']


In [71]:
data['StoreType'].fillna(data['StoreType'].mode()[0], inplace=True)

In [72]:
data['log_competition_distance'].fillna(data['log_competition_distance'].median(), inplace=True)

In [73]:
data.shape

(844338, 22)

In [74]:
data.head()

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,SchoolHoliday,StoreType,Assortment,...,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval,Sales_per_Customer,Month,Year,Day,log_competition_distance,StateHoliday_tmp
0,1,5,2015-07-31,5263,555,1,1,1,3.0,1,...,0,0.0,0.0,0,9.482883,7,2015,31,7.146772,0
1,2,5,2015-07-31,6064,625,1,1,1,1.0,1,...,1,13.0,2010.0,"Jan,Apr,Jul,Oct",9.702400,7,2015,31,6.345636,0
2,3,5,2015-07-31,8314,821,1,1,1,1.0,1,...,1,14.0,2011.0,"Jan,Apr,Jul,Oct",10.126675,7,2015,31,9.556055,0
3,4,5,2015-07-31,13995,1498,1,1,1,3.0,3,...,0,0.0,0.0,0,9.342457,7,2015,31,6.429719,0
4,5,5,2015-07-31,4822,559,1,1,1,1.0,1,...,0,0.0,0.0,0,8.626118,7,2015,31,10.305948,0


In [75]:
# Create variable log_sales for further analysis
data['log_sales'] = np.log(data['Sales'])

## RMSPE function

In [76]:
def RMSPE(y, y_hat):
    return np.sqrt(np.mean((y_hat/y-1)**2))

In [77]:
def rmspe(y_true, y_pred):
    '''
    Compute Root Mean Square Percentage Error between two arrays.
    '''
    loss = np.sqrt(np.mean(np.square(((y_true - y_pred) / y_true)), axis=0))

    return loss

## Train-test split for Time Series Analysis

In [78]:
train_size = int(len(data)*0.7)

In [79]:
train, test = data[:train_size], data[train_size:]

In [80]:
train.shape

(591036, 23)

In [81]:
test.shape

(253302, 23)

# Linear Regression

In [139]:
# train and test data will be created using sklearn's train_test_split
from sklearn.linear_model import LinearRegression

## Linear Regression - train_test_split

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
tmp_cols = ['DayOfWeek', 'Promo', 'Store',
           'SchoolHoliday', 'StoreType', 'Assortment', 'CompetitionOpenSinceMonth',
           'CompetitionOpenSinceYear', 'Promo2', 'Promo2SinceWeek',
           'Promo2SinceYear', 'Month',
           'Year', 'Day', 'log_competition_distance']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(data[tmp_cols], data['Sales'], test_size=0.3, random_state=69)

## Linear Regression - Sales variable

In [142]:
ln_model = LinearRegression(normalize=True)

In [143]:
ln_model.fit(X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=True)

In [144]:
# predictions on training data
y_hat = ln_model.predict(X_train)

# error on training data
print(f"RMSPE on train data: {RMSPE(y_train, y_hat)}")

RMSPE on train data: 0.5241917559538115


In [145]:
# prediction on test data
y_hat = ln_model.predict(X_test)

# error on test data
print(f"RMSPE on test data: {RMSPE(y_test, y_hat)}")

RMSPE on test data: 0.49721006356991565


## Linear Regression - log_sales variable

In [147]:
ln_model = LinearRegression(normalize=True)

In [152]:
ln_model.fit(X_train, np.log(y_train))

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [153]:
# predictions on training data
y_hat = ln_model.predict(X_train)

# error on training data
print(f"RMSPE on train data: {RMSPE(y_train, y_hat)}")

RMSPE on train data: 0.9984956855073233


In [154]:
# prediction on test data
y_hat = ln_model.predict(X_test)

# error on test data
print(f"RMSPE on test data: {RMSPE(y_test, y_hat)}")

RMSPE on test data: 0.9984967513503843


# XGBoost

In [136]:
# train and test data will be created using sklearn's train_test_split

In [108]:
# import XGBRegressor library
from xgboost.sklearn import XGBRegressor

## XGBoost - train_test_split

In [109]:
from sklearn.model_selection import train_test_split

In [110]:
tmp_cols = ['DayOfWeek', 'Promo', 'Store',
           'SchoolHoliday', 'StoreType', 'Assortment', 'CompetitionOpenSinceMonth',
           'CompetitionOpenSinceYear', 'Promo2', 'Promo2SinceWeek',
           'Promo2SinceYear', 'Month',
           'Year', 'Day', 'log_competition_distance']

In [111]:
X_train, X_test, y_train, y_test = train_test_split(data[tmp_cols], data['Sales'], test_size=0.3, random_state=69)

In [112]:
def rmspe_xg(y, y_hat):
    y = np.expm1(y.get_label())
    y_hat = np.expm1(y_hat)
    return "rmspe", rmspe(y,y_hat)

## XGBoost - Sales variable

In [113]:
xgb_model = XGBRegressor()

In [114]:
xgb_model.fit(X_train, y_train)

/Users/mstevankovic/anaconda3/lib/python3.7/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


[22:39:10] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0,
             importance_type='gain', learning_rate=0.1, max_delta_step=0,
             max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
             n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
             silent=None, subsample=1, verbosity=1)

In [116]:
# predictions on training data
y_hat = xgb_model.predict(X_train)

In [117]:
# error on training data
RMSPE(y_train, y_hat)

0.4881520519990625

In [118]:
# prediction on test data
y_hat = xgb_model.predict(X_test)

In [119]:
# error on test data
RMSPE(y_test, y_hat)

0.44728298236866176

## XGBoost - log_sales variable

In [ ]:
xgb_model = XGBRegressor()

In [120]:
xgb_model.fit(X_train, np.log(y_train))

[22:48:21] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0,
             importance_type='gain', learning_rate=0.1, max_delta_step=0,
             max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
             n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
             silent=None, subsample=1, verbosity=1)

In [121]:
# predictions on training data
y_hat = xgb_model.predict(X_train)

In [122]:
# error on training data
RMSPE(y_train, np.exp(y_hat))

0.4223783633675941

In [123]:
# prediction on test data
y_hat = xgb_model.predict(X_test)

In [124]:
# error on test data
RMSPE(y_test, np.exp(y_hat))

0.3884235719928909

## XGBoost - optimized on Sales variable

In [131]:
# instantiate XGBRegressor class
xgb_opt = XGBRegressor(
    n_jobs = -1,
    n_estimators = 1000,
    eta = 0.1,
    max_depth = 2,
    min_child_weight = 2,
    subsample = 0.8,
    colsample_bytree = 0.8,
    tree_method = 'exact',
    reg_alpha = 0.05,
    silent = 0,
    random_state = 1023
)

In [132]:
xgb_opt.fit(X_train, y_train)

[23:19:58] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=0.8, eta=0.1, gamma=0,
             importance_type='gain', learning_rate=0.1, max_delta_step=0,
             max_depth=2, min_child_weight=2, missing=None, n_estimators=1000,
             n_jobs=-1, nthread=None, objective='reg:linear', random_state=1023,
             reg_alpha=0.05, reg_lambda=1, scale_pos_weight=1, seed=None,
             silent=0, subsample=0.8, tree_method='exact', verbosity=1)

In [134]:
# predictions on training data
y_hat = xgb_opt.predict(X_train)

# error on training data
print(f"RMSPE on train data: {RMSPE(y_train, y_hat)}")

RMSPE on train data: 0.41935905176074434


In [135]:
# prediction on test data
y_hat = xgb_opt.predict(X_test)

# error on test data
print(f"RMSPE on test data: {RMSPE(y_test, y_hat)}")

RMSPE on test data: 0.3720715540266975


## XGBoost - optimized on log_sales variable

In [128]:
# instantiate XGBRegressor class
xgb_opt = XGBRegressor(
    n_jobs = -1,
    n_estimators = 1000,
    eta = 0.1,
    max_depth = 2,
    min_child_weight = 2,
    subsample = 0.8,
    colsample_bytree = 0.8,
    tree_method = 'exact',
    reg_alpha = 0.05,
    silent = 0,
    random_state = 1023
)

/Users/mstevankovic/anaconda3/lib/python3.7/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


[23:08:00] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=0.8, eta=0.1, gamma=0,
             importance_type='gain', learning_rate=0.1, max_delta_step=0,
             max_depth=2, min_child_weight=2, missing=None, n_estimators=1000,
             n_jobs=-1, nthread=None, objective='reg:linear', random_state=1023,
             reg_alpha=0.05, reg_lambda=1, scale_pos_weight=1, seed=None,
             silent=0, subsample=0.8, tree_method='exact', verbosity=1)

In [ ]:
xgb_opt.fit(X_train, np.log(y_train))

In [129]:
# predictions on training data
y_hat = xgb_opt.predict(X_train)

# error on training data
print(f"RMSPE on train data: {RMSPE(y_train, np.exp(y_hat)}")

0.36327513822931334

In [130]:
# prediction on test data
y_hat = xgb_opt.predict(X_test)

# error on test data
RMSPE(y_test, np.exp(y_hat))

0.32080568167602996

XGBoost model with the very best results on both train and test data is  
XGBRegressor(  
      n_jobs = -1,  
      n_estimators = 1000,  
      eta = 0.1,  
      max_depth = 2,  
      min_child_weight = 2,  
      subsample = 0.8,  
      colsample_bytree = 0.8,  
      tree_method = 'exact',  
      reg_alpha = 0.05,  
      silent = 0,  
      random_state = 1023  
)  
and this model will be used as XGBoost representative